# Project Report

## Basics of Mobile Robotics, MICRO-452

### Mehdi Krichen, Francesco Nonis, Michael Richter, Vincent Philippoz, Nathan Decurnex

# 1) Environment

Our environment in which our thymio will evolve is a rectangle space composed of two A0 sheet. The size was choosen so that the Thymio could perform efficient and visible local avoidance moves.
The background is green to imitate grass. It has been set to a uniform dark green to not intefer with the thymio's ground sensors (to contrast with the yellow of the crossroads).
The world is also compose of roads and crossroads. The roads will be seen as obstacles. Obviously, the robot can use the crossroads to pass roads. The design of these modules stay simple: rectangular shapes. This is crucial to simplify the vision part. The crossroads were initially yellow and gray like real ones. But we further choose to change them to uniform yellow. This simplified the ground sensors measurements to update the filter, but made their detection in the vision part more difficult. 

The size of the crossroads and road were chosen to approximately be squares of Thymio's dimensions. At the beginning, we wanted to have another Thymio following the white line in the middle of the road. But the main part of the project took us more time than planned.

![env](img.jpg)

# 2) Implementation

## 2.1) Vision

The vision part of the project is essentially used at initialization. It will be used to build an initial obstacle map of the environment. The processing will also find the Thymio initial location and orientation, and the target to reach.

At the start, a picture of the whole environment is taken. The first step is to trim the initial image to obtain a frame with only the environment. We first tried to found the 4 corners of the map in order to perform a reshaping to account for perspective. But the task of finding the corners was not so simple for several reasons. We tried to have one particular shape on each corner so that we could perform template matching with it. But it was hard to detect 4 time the same shape in our environmental conditions; even by filtering our image, other shapes in our environment had regularly more correlation with the template than the real one. We thus thought to change the shape for each corner, but we were limited in the choice of easy distinguishable colors and shape. Moreover, choosing a particular shape could make hard the template detection due to perspective. It why we wanted to have circles. A last factor that made hard edge detection via templatematching is that we do not know the exact size of the template on the image; template matching is very sensitive to the size of the object we are trying to match. In other word, to make work correctly template matching, we should know in advance the size in pixels of our template on the image.  

Before trying some other fancy techniques, we wanted to have a backup. It was to simply mask the green background region of our environment via template matching. This allowed us to have a rather good cropping of the picture as long as it has been taken far away enough; to avoid having straight line deformations. The result was good enough to keep it. We could have tried line detection, but the roads or the floor pattern outside of our map would have also been detected. Sometime to stay simple is the good solution.  

Once the image trimmed, Template matching is again used to detect all the objects on the map. The 4 patterns of turning roads (4 orientations) and the two orientation of straight lines were used to detect the roads. Even with such a simple design, the sensitivity and selectivity of template matching was quite poor. Luminosity is also a decisive factor when trying to detect objects. A lot of hyperparameter needed to be tuned in order to achieve a sufficient road detection. We could have gone for simple uniform gray roads: it would have simplified the template search (also faster computation) and improved the selectivity/sensitivity. But at the beginning, a second robot was planned to follow the road: the central white line would have been used to track the path. 

The crossroads were detected by the same way as the roads. As expected, the result is also highly depending on the tuning of hyperparameters and lighting. The target position was finally decided to be a simple circle. This allowed no orientation dependence: allowing one single template search.

Finally, the Thymio robot is detected thanks to two very small circle placed strategically at the back and front of the robot. Once the circles detected by template matching, a simple calculation is done to obtain the robot hole's position and orientation. We also tried a data driven method: Convolutionnal Neural Network (CNN) and Support Vector Regression to extract the robot position from the image as input. The major issue was to collect data and label it. Simple trials with 25 data points yielded unsatisfying results and stopped our search in this direction. 


### Illustrations

In this section, we present sub-optimal results and particularities that can be observed from.

![env](images/angle1.jpg)

![env](images/angle12.jpg)

Above are illustrations on how sensitive the template matching, for a the road angle, is. The results for the four different orientation yielded the same kind of results showed above. This may be due to deformation of roads on the bottom of the image. The template matches better straight roads than certain correct road on the bottom.

This situation only happens with real big images. When we were testing on smaller scaled map or screenshots of the map on the computer, this did not happen. This shows how image processing becomes harder in real conditions. We think that the template matching could have been improved a little bit: by taking the template image with the same camera that takes the real pictures shown. Currently, the template are the computer image (numerical ones).

This problem was essentially solved by using the high sensitivity of the template matching of the straight roads. This in some way is a bit cheating, but as long as it works...
It was even our final way to find all the roads because it was the more robust we found. In fact the angle roads detection is useless at the end.

![env](images/rd2.jpg)

![env](images/road44.jpg)

We can see at which point, even the wrong orientation of the road can be detected easily. Some crossroads are also captured in the process. Even some opposite orientation roads are detected while the correct orientation is not (see second illustration on the bottom, the template orientation can be deduced: horizontal roads). 

![env](images/psg2.jpg)

Note the crossroad design was change finally.
Of course the same effect appears also for crossroads. The right crossroad is much more detected than the left one. If we want absolutely to detect the two horizontal crossroad, we need to decrease the threshold. This has a direct effect on the right crossroad: it appears larger; even some portion of the right crossroad combined to the road is more detectable than the left crossroad... 

The lighting is even more affecting the results than previously. We think it is because when the yellow is well lighted, it is almost seen like white from the camera. The figure shows a little bit this effect for the crossroad on the left of the image that appears brighter than the other ones.

We finally decided to change the design of the crossroad to simplify the filter localization; uniform yellow. But it made their detection much harder. Finally, as the crossroads had no more white line in the middle, the road template matching could not anymore find them. So we could use only the straight lines pattern to extract all the road and by extent the crossroads; where the roads are not detected it is seen like a hole where the robot is allowed to pass.

### Real Code testing

If you want a real test implementation of the Image Processing part, you can run the cell below. It will print the current progression of the process and show several plots showing each step.

In [ ]:
%run IP/IP_methods.ipynb 
(x,y), orientation = pipeline_IP(plot=True)
print(f'position of thymio is ({x},{y}), the x position is on the horizontal axis and y on the vertical axis. The zero is on the bottom left')
print(f'its orientation is {orientation} [rad] (defined as [0,2Pi] with respect to the horizontal axis)') 

Globally, image processing is very hard. Robust method to lightning, object orientation, deformation or camera type etc.. do not really exist. All lot of testing needed to be done.

## 2.2) Path Planning

Many algorithms exist for path planning. The one used in this project is the vision graph method. It has many disadvantages when building the map, but once it is built, the computation is very quick. Other algorithms were explored, but in an effort to shorten the report, they are explored in the _PathPlanning.ipynb_ notebook.

The first step is to define a coordinate system. We choose an arbitrary corner as the origin and choose the max axis coordinates to be the physical dimensions of our map in mm:
> 2*A0 : 1682 x 1188 mm

Once chosen, we can read the CSV file of the obstacles produced by the vision module of the project. First, the image is read, then saved for ease of use and easy access. 

## 2.3) Global Navigation

## 2.4) Local Navigation

## 2.5) Localization
### How the localization works
#### Introduction
The localization module objective is to estimate the Thymio's pose during the execution of the other modules. To achieve this goal, the module implements a particle filter which uses the motors speeds and the the ground proximity sensors. It proceeds in two steps : Odometry and Measurement.

#### Odometry 
The odometry does the apriori estimation of the position by using the motors speeds and a model of the Thymio. The model is taken from the book *Elements of Robotics* by *Mordechai Ben-Ari and Francesco Mondada*, chapter 5.6.

<img src='images_for_report/odometry_from_book.png' width="400">

It is assumed that we know $b$ the wheelbase, $d_l$ and $d_r$ the distance traveled by the left and right wheels respectively, $(x,y,\phi)$ the initial pose. We want to compute the new pose $(x',y',\phi')$. 

First, we compute $\theta$, the angle of the arc followed by the robot. As the displacement is considered small, we can approximate $sin(x) \approx x$. This means that we get $\theta = \frac{d_r-d_l}{b}$.

Now we can find the vertical and horizontal displacements with these equations:

$dx = \frac{dl+dr}{2} cos(\phi_0 + \frac{\theta}{2})$ ; $dy = \frac{dl+dr}{2} sin(\phi_0 + \frac{\theta}{2})$

Finally, the new pose is given by:

$(x_1,y_1,\phi_1) = (x_0 + dx, y_0 +dy, \phi_0+\theta)$

In the context of the map, the robot position is defined as follow:

<img src='images_for_report/thymio_top_schema.png' width="150">

This image represents the Thymio seen from above. The dot represents the origin of the Thymio, the two grey rectangles are the ground sensors and the black rectangles are the wheels. The robot evolves in the same system of coordinates as the map.

#### Measurement
To add a feedback on the apriori odometry, the ground sensors measure the color of the map below the Thymio. This information is compared to the expected color given by the map.

#### Why a particle filter ?
At first, we wanted to implement a Kalman filter. However, as the Thymio's dynamics are non-linear, this solution was not feasible. The particle filter was chosen because of its robustness, execution speed and relative ease of implementation. Also, we thought it was interesting to implement this type of filter, as it was not done during the exercises.

### Particle filter
*A tutorial on how to implement a particle filter is presented [here](https://salzis.wordpress.com/2015/05/25/particle-filters-with-python/). Also, [this YouTube video](https://www.youtube.com/watch?v=aUkBa1zMKv4&ab_channel=AndreasSvensson) explains the basic principle of a particle filter.*

The particle filter pseudocode is presented below:
```python
def particle_filter_one_step(particles, meas, Pt, Ps):
    '''
    Short: returns a set of particles for the next time step
    
    Inputs: 
    - particles: a vector of particles positions
    - meas: the new incoming measurement
    - Pt: a transition model Pt(new_position | old_position) (probability density function)
    - Ps: a sensor model Ps(measurement | position) (probability density function)

    Local variables: 
    - W: a vector of weights of the same size as Particles
    '''
    N = len(Particles) # number of particles
    
    for i in range(N):
        # step 1: Transition for each particle
        new_position = sample form Pt(new_position | particles[i])
        particles[i] = new_position

        # step 2: Assign weight to particles given the measurement
        W[i] = Ps(meas | particles[i])

        # step 3: Resample the particles
        particles = resample(N, particles, W)
    return [particles, W]

particles = initial position of the robot
while robot is moving:
    old_robot_position = robot.position
    # robot moves
    meas = robot.measure()
    Pt = robot.Pt
    Ps = robot.Ps
    [particles, Weights] = particle_filter_one_step(particles, meas, Pt, Ps)
    robot.position = weighted_mean(particles, Weights)
    robot.direction = weighted_vectorial_sum(particles)
```

The **transition** step is implemented using the ```_apriori_odometry``` function from the ```Moving_point``` class. To simulate the input noise (motor noise), the speed considered for the odometry is a random value generated by a gaussian. Its mean is the measured motor speed, and its standard deviation represents the measured speed noise (which is a hyperparameter).

Each particles is assigned a **weight** which represents how likely it is that the particle is at the same spot than the real robot. This weight is the probability of the real measurement being generated by the particle. For example, if the ground sensor measures "320" and the expected measure at the particle location is "120", then the weight would be very small as it is very unlikely that the particle is at the same spot as the robot.

A key part of the code is the **resampling**. step. In our case, it is simply implemented with a weighted random selection. As it makes no sense to resample particles of similar likelihood, the resampling is done only when the particles have diverse weights.



Finally, once the particle filter step is done, the particles data is used to **estimate the robot pose**. The estimated position is the weighted average of the particles, and the estimated orientation is the vectorial weighted sum of all the particles directions.
To implement the particle filter in a clean way, a system of classes is used.

### The system of classes
The localization module consists of 3 classes:
* ```Moving_point```: This is the base class. It defines the main functions.
* ```Particle```: This class inherits the properties of ```Moving_point``` and adds some methods adapted to particles.
* ```Pose```: This class inherits the properties of ```Moving_point``` and adds some methods adapted to a global point of view. This is the class used for interfacing with the other modules.

#### Moving_point
The ```Moving_point``` class entails the properties of an ideal Thymio robot. It can memorises its pose, do apriori odometry and check if the ground proximity sensors are out of the map. Also, getter and setter functions are provided to interface it if necessary.

#### Particle
The ```Particle``` class is used to simulate a particle of the particle filter. As a particle is simply an ideal Thymio, the ```Particle``` class inherits all the attributes of the ```Moving_point``` class. Furthermore, the particle is given a weight and some methods to compute it. The apriori odometry is also a bit modified to simulate the motor noise. Finally, a plotting function is added.

#### Pose
The ```Pose``` class is in charge of running the particle filter and processing the results. To do so, it is itself a ```Moving_point``` with some extra attributes and methods. These new properties allow the ```Pose``` class to simulate the particles, compute their weights, resample and estimate the position of the real Thymio. All of this is done with one call of the function ```update```.

### Hyperparameters : How to make the code work on my Thymio ?
Every Thymio has unique characteristics. Therefore, one must calibrate some hyperparameters to expect good performances of the odometry. The hyperparameters to tune are :
* ```SPEED_STD```: The standard deviation of each motor
* ```PROX_L_STD```: The standard deviation of the left ground sensor
* ```PROX_R_STD```: The standard deviation of the right ground sensor
* ```FEATURE2PROX_L```: The mean value read by the left ground sensor for each color
* ```FEATURE2PROX_R```: The mean value read by the right ground sensor for each color
* ```SPEED_CONVERSION_FACTOR```: The conversion factor to go from Thymio units to mm/s.

To tune all these values, the next procedures have to be followed.

#### Procedure 1: Movements 1
To make the movement calibration, you can run the code below. It will set ```SPEED_STD```, ```PROX_L_STD``` and ```PROX_R_STD```.

```python
speeds_l = []
speeds_r = []
proxs_l = []
proxs_r = []

for s in range(len(speed_to_test)):
    # Start the motors
    motor_left_target = speed
    motor_right_target = speed
    
    # Measure the speeds
    for i in range(nb_calibration_samples):
        speeds_l.append(motor_left_speed)                
        speeds_r.append(motor_right_speed)
        proxs_l.append(prox_ground_delta[0])
        proxs_r.append(prox_ground_delta[1])
        sleep(period)
    
    # Stop the motors
    motor_left_target = 0
    motor_right_target = 0
    
# Compute the standard deviations
SPEED_STD[0][str(s)] = np.std(speeds_l)
SPEED_STD[1][str(s)] = np.std(speeds_r)
PROX_L_STD = np.std(proxs_l)
PROX_R_STD = np.std(proxs_r)
```

#### Procedure 2: Colors
To make the color calibration, you can run the code below. It will set ```FEATURE2PROX_L``` and ```FEATURE2PROX_L```.
```python
for color in colors_to_test:
    proxs_l = []
    proxs_r = []

    # Measure the color
    print(f"Place the robot on '{color}' in the next 5 seconds.")
    sleep(5) # To have time to move the robot
    for i in range(nb_calibration_samples):
        proxs_l.append(prox_ground_delta[0])
        proxs_r.append(prox_ground_delta[1])
        sleep(period)

    # Compute the mean measurement
    FEATURE2PROX_L[color] = np.mean(proxs_l)
    FEATURE2PROX_R[color] = np.mean(proxs_r)
```

#### Procedure 3: Movements 2
Only one hyperparameters has to be tuned by hand : ```SPEED_CONVERSION_FACTOR```. To do so, place the Thymio on the map, with a straight line access to its objective. Run the global code until the Thymio stops. If it overshot the goal, increase the  ```SPEED_CONVERSION_FACTOR```. If it undershot, do the opposite. Repeat the process until the results are satisfying.

### Interfacing : How to use the localisation module ?
To run the odometry, one must first create an instance of the class Pose:
```python
myPose = Pose(x=x0, y=y0, theta=theta0)
```
You can also set ```isLost``` and ```nb_particles```, but they are not mandatory.


Then, in your code where you make the thymio move, you call the method ```update```:

```python
myPose.update(speed, prox, dt, myMap)
```
This functions does everything for you, it updates the inner variables automatically. You can see in details what are the arguments by using ``` help(Pose.update)```. You can also do this for any function of any class.

If you want to know where the robot is, call the method ```myPose.get_coords()```. It returns the pose as a dict.

You can also set the pose using ```myPose.set_coords(args)```. In ```args```, you can either put ```x, y, theta``` or a dict ```{'x':x, 'y':y, 'theta':theta}```. This way, this code works :
```python
myPose.set_coords(x0, y0, theta0)
# do something with myPose
myCoords = myPose.get_coords()
# do something with myCoords
myPose.set_coords(myCoords)
```

The pose can be plotted using ```myPose.plot()```. Optional boolean arguments include ```plot_particles``` and ```plot_theta```.


In the cell below, you can test a working example of simulation. It assigns randomly the speeds of the motors and plots the pose every 5 iterations.

For it to work, one must open a valid map called myMap in the same notebook.

In [ ]:
MAX_SPEED = 560
MIN_SPEED = 100
MAX_PROX = 1000
DT = 0.2
NB_PARTICLES = 100


for i in range(40):
    speed = [random.randrange(MIN_SPEED, MAX_SPEED), random.randrange(MIN_SPEED, MAX_SPEED)]
    prox = myPose._expected_prox(myMap)
    
    # add noise on the prox
    prox[0] += random.gauss(0, PROX_L_STD)
    prox[1] += random.gauss(0, PROX_R_STD)

    myPose.update(speed, prox, DT, myMap)
    
    # plot only 1 on 5 steps
    if i%5 == 0:
        myPose.plot(plot_theta=False, theta_size=5, plot_particles=True)
    
plt.scatter(0, 0, marker='+')
plt.xlim(0, MAP_REAL[0])
plt.ylim(0, MAP_REAL[1])
plt.show()

In [ ]:
help(Pose.plot)

# 3) Conclusion 